In [35]:
import tensorflow as tf

import numpy as np
import os
import time

In [36]:
path_to_file = 'trump_18June2020.txt'

In [37]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')


Length of text: 176072 characters


In [40]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [42]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [43]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[54, 55, 56, 57, 58, 59, 60], [77, 78, 79]]>

In [46]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [47]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [48]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(176072,), dtype=int64, numpy=array([ 1, 40, 61, ..., 58, 53,  1], dtype=int64)>

In [49]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [50]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))



O
h
,
 
w
e
 
h
a


In [51]:
seq_length = 100

In [52]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\n' b'O' b'h' b',' b' ' b'w' b'e' b' ' b'h' b'a' b'd' b' ' b's' b'u'
 b'c' b'h' b' ' b'l' b'u' b'c' b'k' b' ' b'i' b'n' b' ' b'O' b'r' b'l'
 b'a' b'n' b'd' b'o' b',' b' ' b'w' b'e' b' ' b'l' b'o' b'v' b'e' b' '
 b'b' b'e' b'i' b'n' b'g' b' ' b'i' b'n' b' ' b'O' b'r' b'l' b'a' b'n'
 b'd' b'o' b'.' b' ' b'T' b'h' b'a' b'n' b'k' b' ' b'y' b'o' b'u' b'.'
 b' ' b'T' b'h' b'a' b'n' b'k' b' ' b'y' b'o' b'u' b',' b' ' b'O' b'r'
 b'l' b'a' b'n' b'd' b'o' b'.' b' ' b'W' b'h' b'a' b't' b' ' b'a' b' '
 b't' b'u' b'r'], shape=(101,), dtype=string)


In [53]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\nOh, we had such luck in Orlando, we love being in Orlando. Thank you. Thank you, Orlando. What a tur'
b'nout! What a turnout! You know this is a very big arena for a Tuesday night. I said, you know, if we '
b'have about three or four empty seats, the fake news will say, Headlines: "He didn\'t fill up the arena'
b'", so I said, maybe we shouldn\'t take a chance.\nMaybe we shouldn\'t go to Orlando as you go someplace '
b'else, then I said, "No, I think, we\'ll go to Orlando." And -- and not only did we fill it up, but we '


In [54]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [55]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [56]:
dataset = sequences.map(split_input_target)

In [57]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\nOh, we had such luck in Orlando, we love being in Orlando. Thank you. Thank you, Orlando. What a tu'
Target: b'Oh, we had such luck in Orlando, we love being in Orlando. Thank you. Thank you, Orlando. What a tur'


In [58]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [59]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [60]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [61]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [62]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 84) # (batch_size, sequence_length, vocab_size)


In [63]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     multiple                  21504     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense_3 (Dense)             multiple                  86100     
                                                                 
Total params: 4,045,908
Trainable params: 4,045,908
Non-trainable params: 0
_________________________________________________________________


In [64]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [65]:
sampled_indices

array([66, 36, 23,  8, 51, 79, 48,  0, 29, 73, 27, 57, 25, 78,  8, 62, 72,
       72, 60,  0, 21, 63, 15, 32, 36, 49, 56, 65, 27, 75, 45, 52, 73,  8,
       65, 16, 24, 25, 76, 27, 76, 83, 40,  9, 36, 21, 63, 49, 11,  5, 11,
       20, 54, 56, 17, 28, 70, 20, 25, 46, 55, 41, 48, 55, 53,  4, 46, 15,
        6, 62, 38, 20, 36,  8, 26,  1, 49, 51, 11, 36, 78, 61, 56, 65, 83,
       82, 67,  2, 62,  7, 77, 65, 42, 38, 70, 19, 78,  0, 22, 15],
      dtype=int64)

In [68]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"er china's not exactly happy with being billions, they pay us billions and billions of dollars and t"

Next Char Predictions:
 b'mK:(ZzW[UNK]DtBd?y(issg[UNK]8j2GKXclBvT[t(l3;?wBw\xc3\x83O)K8jX-$-7ac4Cq7?UbPWb]"U2%iM7K(A\nXZ-Kyhcl\xc3\x83\xc2\xban i\'xlQMq6y[UNK]92'


In [69]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [70]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 84)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.4297676, shape=(), dtype=float32)


In [71]:
tf.exp(example_batch_mean_loss).numpy()

83.91191

In [72]:
model.compile(optimizer='adam', loss=loss)

In [73]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [76]:
EPOCHS = 1


In [78]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


27/27 [==============================] - 58s 2s/step - loss: 3.2068


In [79]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [80]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [81]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:meAte s sobnso aoeko nspe ptesl,t slofegmaangpedg crl iite d tleuawe tAiadfsf,in yn'dow oetiute't inllryf la nn rt udp iaorgou tr Ct ags tl atnant't denwtea je e teev urme ies nllemto. hhnioe tne goigy ao eno'o o ioapceay lcepeyelneigdvWgs s iwenss t r.ooge YhoCy met anhn hy r'yvmclue   acededun thnoan.vpllgutkao etei  u oyue riren nei,hitinspwedm eTnedolg rlm seiltpipdiaIn lee a e tadomitw. ahhehbtrn edrtsy rishnt bot aleo, I o hTnnn o katnao8Iitppmacg'ae m,a u.se cro obeauii ysr y ,faaeeeil "ulwfnan antf e it. toreiipnaaKrgdstlh wt uols a d rog iimhleantga,s wrre 0r, 6l e hdoaerw psscaly uehs au anvnas lopen,nhy eim te ooren e dt sy l l,iltpofna f tttth-l, ferhy r hoyibt a aosgeotay iodu aTh sar oouye he eA..ibstdr hecryn h. vWly.oWnlgoaun ntToe t yRuirl riknaktfsto t nire, Yea w at iaott d.mhe tor,n a oa h oseuyon olece Dunrhrhcnseeyt wlrl w, bohe teegatnmhhitfa' er nain ieas,d irni ohkhsftse lngidn oht aa mwe utouhcy slpe costI lso acy un  ,nencers tuAI iny he pe inal e iaali